# 0. Colab 기본 세팅

In [ ]:
# 구글 드라이브 마운트(cjyjob1993@gmail.com)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# lib 디렉토리를을 환경 변수에 추가(lib 내 파일 변경 시, 런타임을 재시작 해주어야 변경 내용이 적용됨.)
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/CodeStates/CodeStates_Project_1/lib')

In [ ]:
# debug를 위한 출력 모듈 임포트 (config.py의 debug_flag가 1이면, 디버그 내용 출력.)
import debugLog
from debugLog import debugLog
debugLog('start debug')

debug_log | 2023.01.07 05:50:00 | >>> start debug


# 1. 인공신경망 구현하기

## [0] 기본 라이브러리 임포트

In [ ]:
# 사용 라이브러리 임포트
debugLog('기본 라이브러리 임포트')
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

debug_log | 2023.01.07 05:50:00 | >>> 기본 라이브러리 임포트


In [ ]:
# dataframe 을 확인 시 편의를 위해, 출력 column 수 제한을 없앰
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## [1] 데이터 불러오기 기능

In [ ]:
def dataLoad() :
  debugLog(f'dataLoad() 를 시작합니다.')
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CodeStates/CodeStates_Project_1/binary_dataset.csv')
  debugLog(f'data 를 df 변수로 바인딩했습니다.')
  debugLog(f'dataLoad() 를 종료합니다. 반환값은 \n\tdf = \n{df} \n 입니다.')
  return df

## [2] 파라미터와 편향 생성 기능


In [ ]:
def createPerceptron(df) :
  debugLog(f'createPerceptron() 를 시작합니다.')
  featuresNum = len(df.columns) - 1
  debugLog(f'입력받은 df의 feature 개수를 featuresNum 변수에 바인딩합니다.')
  weights = np.random.rand(featuresNum)
  debugLog(f'featuresNum의 수 만큼 weight 값을 난수(0~1 사이의 수)로 초기화합니다. 해당 값을 weights 변수로 바인딩합니다.')
  bias = np.random.rand(1)[0]
  debugLog(f'perceptron의 outputNode(1개) bias 값을 난수(0~1 사이의 수)로 초기화합니다. 해당 값을 bias 변수로 바인딩합니다.')
  debugLog(f'craetePerceptron() 을 종료합니다. 반환값은 \n\tWeights = {weights}, \n\tBias = {bias}) 입니다.')
  return weights, bias

## [3] 데이터 뒤섞기 기능


In [ ]:
def shuffleDf(df) :
  debugLog(f'shuffleDf() 를 시작합니다.')
  idx_i = df.index
  df = df.sample(frac=1) # shuffling
  idx_f = df.index
  df = df.reset_index(drop=True) # index reset
  debugLog(f'df를 shuffle하고 index를 초기화해 df로 재할당 합니다. 처리 전후 idx는 다음과 같습니다. \n\tidx_i : {list(idx_i)} \n\tidx_f : {list(idx_f)}')
  debugLog(f'shuffleDf() 를 종료합니다. 반환값은 shuffle된 df 입니다.')
  return df

## [4] 데이터 전처리

### [4-1] 데이터 라벨, 특징 분리 기능

In [ ]:
def divideLabelFeature(df, label, features) :
  debugLog(f'divideLabelFeature() 를 시작합니다.')
  X = df[features]
  debugLog(f'features에 해당하는 데이터를 X 변수로 바인딩합니다.')
  y = df[label]
  debugLog(f'label에 해당하는 데이터를 y 변수로 바인딩합니다.')
  debugLog(f'divideLabelFeature() 를 종료합니다. 반환값은 \n\tfeatures = \n{X} \n\tlabel = \n{y}')
  return X, y

### [4-2] 노말라이제이션

In [ ]:
def normalize(df) :
  debugLog(f'normalize() 를 시작합니다.')
  debugLog(f'각 칼럼의 최댓값으로 열의 값들을 나눠 정규화합니다.')
  result = df.copy()
  debugLog(f'결과값을 기록할 df copy를 생성합니다.')
  for col in df.columns :
    max = df[col].max()
    debugLog(f'{col} column의 최대값을 기록합니다.')
    result[col] = df[col] / max
    debugLog(f'{col} 칼럼의 각 값을 {col} 칼럼의 최댓값으로 나눕니다.')
  debugLog(f'normalize() 를 종료합니다. 반환값은 \n\t df = \n{result}')
  return result

### [4-3] 학습, 테스트 데이터 분리 기능


In [ ]:
def splitData(df, train_portion) :
  debugLog(f'splitData() 를 시작합니다.')
  div = int(len(df) * train_portion)
  debugLog(f'train_portion의 비율대로 데이터를 나눌 수 있도록 index(data 개수 * portion)를 구해, 변수 div(={div})로 바인딩합니다.')
  train = df.iloc[:div]
  debugLog(f'train dataset을 train 변수로 바인딩합니다.')
  test = df.iloc[div:]
  debugLog(f'test dataset을 test 변수로 바인딩합니다.')
  debugLog(f'splitData() 를 종료합니다. 반환값은 \n\ttrain = \n{train} \n\ttest = \n{test} \n입니다.')
  return train, test

## [5] 미니배치를 고려한 학습 데이터 기반의 신경망 연산 및 이진 판단 예측 기능


In [ ]:
def minibatch(df, batchSize) :
  debugLog(f'minibatch()를 시작합니다. 입렵값은 batchSize = {batchSize} \n\t df = \n{df}')
  result = []
  debugLog(f'batchSize에 맞춰 자른 df들을 result 리스트에 append합니다.')
  count = len(df) // batchSize
  for i in range(count) :
    div = i*batchSize
    debugLog(f'{i} minibatch : df[{div}:{div+batchSize}]\n{df[div:div+batchSize]}')
    result.append(df[div:div+batchSize])
  debugLog(f'minibatch()를 종료합니다. 반환값은 각 minibatch를 element로 갖는 리스트 입니다')
  return result  

In [ ]:
def calculForwardProp(df, weights, bias) :
  debugLog(f'calculForwardProp() 를 시작합니다.')

  result = []

  debugLog(f'\n\tweights = \n{weights}')
  debugLog(f'\n\tbias = \n{bias}')
  for index, row in df.iterrows():
    result.append(np.dot(row, weights) + bias)
    debugLog(f'sum({index} input * weight) + bias 의 결과를 reulst 리스트에 append 합니다.')
  
  debugLog(f'calculForwardProp() 를 종료합니다. 반환값은 \n\tweightSum = \n{result}')
  return result

In [ ]:
def sigmoid(x):
  debugLog(f'sigmoid() 를 시작합니다.')
  debugLog(f'sigmoid() 를 종료합니다. 반환값은 입력값({x})의 sigmoid 함수 출력값 ({1 / (1 + np.exp(-x))}) 입니다')
  return 1 / (1 + np.exp(-x))

In [ ]:
def forwardProp(df, weights, bias, batchSize): 
  debugLog(f'forwardProp() 를 시작합니다.')
  batchs = minibatch(df, batchSize)
  result = [calculForwardProp(df, weights, bias) for df in batchs]
  tmp = []
  for weightsSums in result :
    x = []
    for sum in weightsSums:
      x.append(sigmoid(sum))
    tmp.append(x)
  result = tmp
  debugLog(f'minibatch 별로 weightSum을 구해, 하나의 리스트로 묶어 result에 append')
  debugLog(f'forwardProp() 를 종료합니다. 반환값은 \n\tsigmoid(weightsSums) \n{result}입니다.')
  return result

## [6] 손실(교차 엔트로피)값 연산 기능


In [ ]:
def crossEntropyError(y, t):
  debugLog(f'crossEntropyError() 를 시작합니다.')
  debugLog(f'y가 0인 경우 -inf 값을 예방하기 위해, 아주 작은 값인 delta를 선언해 사용한다.')
  delta = 1e-7 #아주 작은 값 더하기 (y가 0인 경우 -inf 값을 예방)
  y = [x + delta for x in y]
  debugLog(f'crossEntropyError()를 종료합니다. 출력값(y, {y}), 예측값(t, {t}) 의 CEE값은 {-np.sum(t*np.log(y))}')
  return -np.sum(t*np.log(y))

## [7] 정확도 연산 기능

In [ ]:
def batchScore(y, t, threshold=0.5) :
  for i in range(len(y)) :
    if y[i] > threshold :
      y[i] = 1
    else :
      y[i] = 0
  
  answer = 0 
  for i in range(len(y)) :
    if y[i] == t.iloc[i] :
      answer += 1

  return answer / len(y) 


## [8] 전체 기능의 동작 

In [ ]:
# 예시) **전체 기능의 동작 메서드 정의**
def main():

  # 데이터 불러오기 기능
  df = dataLoad()

  # Perceptron 생성
  weights, bias = createPerceptron(df)

  # 데이터 뒤섞기
  df = shuffleDf(df)

  # 데이터와 라벨 분리
  label = df.columns[-1]
  features = df.columns[:-1]
  x_df, y_df = divideLabelFeature(df, label, features)
  
  # 데이터 정규화
  x_df = normalize(x_df)

  # train, test dataset 분리하기
  x_train, x_test = splitData(x_df, 0.8)
  y_train, y_test = splitData(y_df, 0.8)

  # 순전파
  batchSize = 4
  result = forwardProp(x_train, weights, bias, batchSize)

  # 손실 계산 
  loss = []
  for batch in range(len(result)):
    loss.append(crossEntropyError(result[batch], y_train[batch*batchSize:(batch+1)*batchSize]))

  # 정확도 계산
  total_score = 0
  for batch in range(len(result)) :
    total_score += batchScore(result[batch], y_train[batch*batchSize:(batch+1)*batchSize])
  
  print(f'loss = {np.mean(loss)}, accuracy = {total_score/len(result)}')

In [ ]:
main()

debug_log | 2023.01.07 05:50:00 | >>> dataLoad() 를 시작합니다.
debug_log | 2023.01.07 05:50:00 | >>> data 를 df 변수로 바인딩했습니다.
debug_log | 2023.01.07 05:50:00 | >>> dataLoad() 를 종료합니다. 반환값은 
	df = 
            x1         x2        x3         x4          x5         x6         x7          x8  y
0   136.093750  51.691005 -0.045909  -0.271816    9.342809  38.096400   4.345438   18.673649  0
1    99.367188  41.572202  1.547197   4.154106   27.555184  61.719016   2.208808    3.662680  1
2   100.890625  51.890394  0.627487  -0.026498    3.883779  23.045267   6.953168   52.279440  0
3   120.554688  45.549905  0.282924   0.419909    1.358696  13.079034  13.312141  212.597029  1
4   121.882812  53.042675  0.200521  -0.282219    2.116221  16.580876   8.947603   91.011762  0
5   125.210938  51.175197  0.139851  -0.385737    1.147993  12.414012  14.068797  228.131554  0
6   141.968750  50.470898  0.244974  -0.342665    2.823579  16.238188   8.207744   85.532584  0
7   136.500000  49.932767  0.044623  -0.37